## Aditya Jindal
## Batch T6
## Assignment6
## Recurrent Neural Network & Long and Short Term Memory

In [2]:
# Importing Libraries

import numpy
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [3]:
#Mounting drive

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# Load ascii text and covert to lowercase

filename = "/content/gdrive/My Drive/EIP3_Weights/sourceLSTMSeed.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()
raw_text = raw_text.replace('\n',' ')

In [5]:
# Create mapping of unique chars to integers
# Removed punctuation marks from source text
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
print(chars)
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

[' ', '0', '3', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
Total Characters:  136183
Total Vocab:  29


In [6]:
# Prepare the dataset of input to output pairs encoded as integers
# and using PrePadding on train data

seq_length = 80
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
dataX = pad_sequences(dataX, maxlen = 100)
print("Total Patterns: ", n_patterns)
print(dataX[1])


# import re
# seq_length = 100
# base_length = 10
# words = re.split(r'\n', raw_text)
# dataX = []
# dataY = []
# for index, line in enumerate(words[:]):
#   print(index)
#   line_size = len(line)
#   if(line_size>0):
#     for i in range(0, line_size-base_length, 1):
#       seq_in = line[i:i + base_length]
#       seq_out = line[i + base_length]
#       dataX.append([char_to_int[char] for char in seq_in])
#       dataY.append(char_to_int[seq_out])
# dataX = pad_sequences(dataX, maxlen = 100)
# n_patterns = len(dataX)
# print("Total Patterns: ", n_patterns)


Total Patterns:  136103
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 14 11  5  7
 21  0  3  6 24  7 16 22 23 20  7 21  0 11 16  0 25 17 16  6  7 20 14  3
 16  6  0  0 14  7 25 11 21  0  5  3 20 20 17 14 14  0  0 22 10  7  0 15
 11 14 14  7 16 16 11 23 15  0  8 23 14  5 20 23 15  0  7  6 11 22 11 17
 16  0  2  1]


In [0]:
# Reshape X to be [samples, time steps, features]

pattern_length = 100
X = numpy.reshape(dataX, (n_patterns, pattern_length, 1))
# Normalize
X = X / float(n_vocab)
# One hot encode the output variable
y = np_utils.to_categorical(dataY)

In [8]:
# Define the LSTM model
input = Input(shape=(X.shape[1], X.shape[2]))

DL1 = Dropout(0.1)(input)

LSTM_L1 = LSTM(256, return_sequences=True)(DL1)

DL2 = Dropout(0.1)(LSTM_L1)

LSTM_L2 = LSTM(256)(DL2)

FCL1 = Dense(y.shape[1], activation='softmax')(LSTM_L2)

model = Model(inputs=[input], outputs=[FCL1])

model.summary()

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

W0725 02:21:20.442817 139623993382784 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0725 02:21:20.481038 139623993382784 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0725 02:21:20.487679 139623993382784 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0725 02:21:20.497370 139623993382784 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a futur

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100, 1)            0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 1)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 256)          264192    
_________________________________________________________________
dropout_2 (Dropout)          (None, 100, 256)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dense_1 (Dense)              (None, 29)                7453      
Total params: 796,957
Trainable params: 796,957
Non-trainable params: 0
_________________________________________________________________


In [0]:
# Define the checkpoint
weights_path="/content/gdrive/My Drive/EIP3_Weights/weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(weights_path, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [13]:
# Training the model  #Finished 50 epochs

model.fit(X, y, epochs=50, batch_size=128, callbacks=callbacks_list)

W0724 11:43:12.444819 139809997686656 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/50
136103/136103 [==============================] - 395s 3ms/step - loss: 2.7095 - acc: 0.2253

Epoch 00001: loss improved from inf to 2.70954, saving model to /content/gdrive/My Drive/EIP3_Weights/weights-improvement-01-2.7095.hdf5
Epoch 2/50
136103/136103 [==============================] - 388s 3ms/step - loss: 2.4386 - acc: 0.2874

Epoch 00002: loss improved from 2.70954 to 2.43864, saving model to /content/gdrive/My Drive/EIP3_Weights/weights-improvement-02-2.4386.hdf5
Epoch 3/50
136103/136103 [==============================] - 388s 3ms/step - loss: 2.2626 - acc: 0.3383

Epoch 00003: loss improved from 2.43864 to 2.26264, saving model to /content/gdrive/My Drive/EIP3_Weights/weights-improvement-03-2.2626.hdf5
Epoch 4/50
136103/136103 [==============================] - 387s 3ms/step - loss: 2.1509 - acc: 0.3715

Epoch 00004: loss improved from 2.26264 to 2.15089, saving model to /content/gdrive/My Drive/EIP3_Weights/weights-improvement-04-2.1509.hdf5
Epoch 5/50
136103/136103

In [0]:
# Load the network weights
load_wt = "/content/gdrive/My Drive/EIP3_Weights/weights-improvement-50-1.4225.hdf5"
model.load_weights(load_wt)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
# Training the model #Finished 25 epochs

model.fit(X, y, epochs=50, batch_size=128, callbacks=callbacks_list)

Epoch 1/50
136103/136103 [==============================] - 405s 3ms/step - loss: 1.4172 - acc: 0.5713

Epoch 00001: loss improved from 1.42248 to 1.41718, saving model to /content/gdrive/My Drive/EIP3_Weights/weights-improvement-01-1.4172.hdf5
Epoch 2/50
136103/136103 [==============================] - 403s 3ms/step - loss: 1.4129 - acc: 0.5717

Epoch 00002: loss improved from 1.41718 to 1.41288, saving model to /content/gdrive/My Drive/EIP3_Weights/weights-improvement-02-1.4129.hdf5
Epoch 3/50
136103/136103 [==============================] - 403s 3ms/step - loss: 1.4057 - acc: 0.5737

Epoch 00003: loss improved from 1.41288 to 1.40573, saving model to /content/gdrive/My Drive/EIP3_Weights/weights-improvement-03-1.4057.hdf5
Epoch 4/50
136103/136103 [==============================] - 403s 3ms/step - loss: 1.4057 - acc: 0.5736

Epoch 00004: loss improved from 1.40573 to 1.40568, saving model to /content/gdrive/My Drive/EIP3_Weights/weights-improvement-04-1.4057.hdf5
Epoch 5/50
136103/13

In [0]:
# Load the network weights
load_wt = "/content/gdrive/My Drive/EIP3_Weights/weights-improvement-23-1.3456.hdf5"
model.load_weights(load_wt)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [11]:
# Training the model #Finished 25 epochs

model.fit(X, y, epochs=25, batch_size=128, callbacks=callbacks_list)

W0725 02:22:54.023031 139623993382784 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/25
136103/136103 [==============================] - 289s 2ms/step - loss: 1.3464 - acc: 0.5882

Epoch 00001: loss improved from inf to 1.34637, saving model to /content/gdrive/My Drive/EIP3_Weights/weights-improvement-01-1.3464.hdf5
Epoch 2/25
136103/136103 [==============================] - 289s 2ms/step - loss: 1.3414 - acc: 0.5901

Epoch 00002: loss improved from 1.34637 to 1.34141, saving model to /content/gdrive/My Drive/EIP3_Weights/weights-improvement-02-1.3414.hdf5
Epoch 3/25
136103/136103 [==============================] - 288s 2ms/step - loss: 1.3389 - acc: 0.5908

Epoch 00003: loss improved from 1.34141 to 1.33893, saving model to /content/gdrive/My Drive/EIP3_Weights/weights-improvement-03-1.3389.hdf5
Epoch 4/25
136103/136103 [==============================] - 286s 2ms/step - loss: 1.3391 - acc: 0.5904

Epoch 00004: loss did not improve from 1.33893
Epoch 5/25
136103/136103 [==============================] - 281s 2ms/step - loss: 1.3349 - acc: 0.5916

Epoch 00005: l

In [0]:
# Load the network weights
load_wt = "/content/gdrive/My Drive/EIP3_Weights/weights-improvement-24-1.2937.hdf5"
model.load_weights(load_wt)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
# Reverse mapping that we can use to convert the integers back to characters 
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [16]:
# Pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
pred_result = []
print("Seed:")
seq_in = ''.join([int_to_char[value] for value in pattern])
print("\"", seq_in, "\"")
# Generate 500 characters
for i in range(500):
  x = numpy.reshape(pattern, (1, len(pattern), 1))
  x = x / float(n_vocab)
  prediction = model.predict(x, verbose=0)
  index = numpy.argmax(prediction)
  result = int_to_char[index]
  pred_result = numpy.append(pred_result, result)
  if(i%100==0):
    pred_result = numpy.append(pred_result, "\n")
  pattern = numpy.append(pattern, index)
  pattern = pattern[1:len(pattern)]
print("Generated Text:\n\"", ''.join(pred_result), "\"")
print("\nDone.")

Seed:
"                     ind that she knew the name of nearly everything there thats the judge she said t "
Generated Text:
" u
itted alice anl e waleny temarkln  tepaied oo  sioly anice dould ho  siog  tie hoan  teparking anice
 anl e wo hrod hor separk anice anl e wane tepain t ie walg tuitted alice anl e walenned anlce anice
 vhought anice anl e whos ht taid tuitted anice anl e whos hot serpent he rnate whe mitt and thaling
 atpealing and the had nearnt poly teparks anl tie havter whos ht  ase tepained tuitlng teally taid 
tuitted anice anl e whos hot serpee  tespent ier anice anl e wnder tholied ier anice sepained tathe "

Done.
